<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>** 
This contains all the data preprocessing and cleaning</p> 

In [1212]:
import os
print(os.getcwd())

/Users/imanteh/f20bc/F21BC/Notebooks


In [1213]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [1214]:
df = pd.read_csv("../Dataset/concrete_data.csv", skiprows=1)

In [1215]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [1216]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [1217]:
df.dropna(inplace = True)
df.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [1218]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [1219]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>** 

In [1220]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [1221]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [1222]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [1223]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [1224]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [1225]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [1226]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x
    
    # def elu(x, alpha=1.0):
    #     return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]
    
    def visualize_layers(self):
        for i in range(len(self.layerSize)-1):  
            print("LAYER COUNT:  ", i,"    NODES: ",self.layerSize[i])
            print(i)
            print(f"  Weights Shape:   ", self.weights[i].shape)
            print(f"  Biases Shape:    ",self.biases[i].shape )
            print(f"  Weights= : {self.weights[i]}")
            print(f"  Biases= {self.biases[i]}")
            
    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)  
        return output

Layer size is structure to be 8,16,8,1 because 
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [1227]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)
ann.visualize_layers()

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

LAYER COUNT:   0     NODES:  8
0
  Weights Shape:    (8, 16)
  Biases Shape:     (1, 16)
  Weights= : [[-0.5344936   0.46558538 -0.03555715 -0.21315328  0.9239487   0.23647274
   0.05400646  0.0673876  -1.54587892  0.32077179  0.30637888  0.40423229
  -0.24032778 -0.2127602   1.0165421   0.40449037]
 [-1.04767111  0.65970553  0.31688568  0.03334888  0.216169   -0.43217275
  -0.62654237 -0.32560627  0.66257553 -0.28331339  1.11647225  0.64283512
  -0.21887563 -0.23605023  0.08151371  0.13885154]
 [-0.32680691  0.34771373  1.14817753 -0.44705896 -0.51638874  0.26900766
  -0.72443459  0.06521146  0.62465312 -0.07172165  0.31161461 -0.07891479
   0.43450633 -1.20678909  0.01857857 -0.45355061]
 [ 0.54297309  0.10049187  0.50104818  0.47581814 -1.38405805  0.32686663
   0.47161121 -0.41028721  0.00572783 -1.05647746 -0.4316316  -0.1413345
  -0.1832205  -0.35268209 -0.45748432  0.40191529]
 [ 0.20205576  0.16599392  0.15648023  1.09135588  0.4259138   0.56651683
  -0.29481159  0.42725051  0.

**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MSE is utilised as the loss function

In [1228]:
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean((y_pred - y_train) ** 2)

In [1229]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN
# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape) 
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss_function=MeanSquaredError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 1264.6379607984675


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [1230]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [1231]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue
        
        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange
        
        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue
        
        # activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [1232]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x) 
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [1233]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, omega, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.omega = omega
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):
        
        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # Update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )

                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition

<p style="font-size:18px;">Test out PSO Functionality</p>

In [1234]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(100, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(100, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO 
swarmSize = 10
alpha = 0.5  # inertia weight
beta = 0.5   # cognitive parameter
delta = 0.5  # social parameter
jumpSize = 0.5
informantCount = 3
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 50

# initialize the PSO optimiser
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    omega = omega,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimisation
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,
    
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    loss_function= MeanSquaredError(),
    max_iterations = max_iterations
)
print("Optimised ANN Parameters:", best_parameters)

Optimised ANN Parameters: [0.12093577 0.57526869 0.61514954 0.83009576 0.28939801 0.85119601
 0.62269397 0.25565307 0.7944192  0.53508049 0.66222201 0.14177281
 0.09399338 0.92001664 0.73986493 0.06586319 0.76183357 0.76915725
 0.63281595 0.2008563  0.2980001  0.89657968 0.15106033 0.42338253
 0.440563   0.19278999 0.1164078  0.43416178 0.22568048 0.03132411
 0.50746962 0.72679913 0.63307539 0.10787054 0.9767174  0.07922487
 0.52721549 0.7645804  0.25834639 0.91260172 0.47120798 0.19446552
 0.76272999 0.74417239 0.42225365 0.92308749 0.3056555  0.72575778
 0.60705158 0.84472964 0.96120522 0.274451   0.10815997 0.59327807
 0.3447657  0.34530596 0.34452063 0.8492009  0.8247442  0.54680711
 0.20690925 0.54102129 0.67700433 0.8327914  0.69212293 0.28398124
 0.21925313 0.65018035 0.69822859 0.17472622 0.14126099 0.55658023
 0.41115474 0.68156593 0.11765282 0.55502095 0.16643141 0.83809661
 0.08454337 0.42556793 0.09497429 0.5493641  0.85843973 0.32955094
 0.65096805 0.35201077 0.16765469 0.

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimised by the PSO

In [1235]:
# create a dummy particle and assign the optimised parameters
optimised_particle = Particle(vectorSize)
optimised_particle.particlePosition = best_parameters

# convert the optimised particle position back to ANN weights and biases
optimised_ann = particleToAnn(
    optimised_particle, 
    annLayers,
    activationFunctions
)

# run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimised_particle,  # pass the optimised particle for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    loss_function = MeanSquaredError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 0.37908212469034924


<p style="font-size:18px;">Training the ANN</p>

In [1236]:
y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)
loss_function = MeanSquaredError()
training_loss = loss_function.evaluate(y_train_pred, y_train)

print(f"Training Loss: {training_loss}")

Training Loss: 0.37908212469034924


<p style="font-size:18px;">Testing the model on the test set</p>

In [1237]:
X_test_scaled = np.random.rand(100, 8)  # Replace with actual test data
y_test = np.random.rand(100, 1)  # Replace with actual test labels

y_test_pred = optimised_ann.forwardPropagation(X_test_scaled)
test_loss = loss_function.evaluate(y_test_pred, y_test)

print(f"Test Loss: {test_loss}")

Test Loss: 0.3434949218047184


In [1238]:
epochs = 500  # Set this to the number of epochs you want
max_iterations = 100 
for epoch in range(epochs):
    y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)
    
    loss_function = MeanSquaredError()
    training_loss = loss_function.evaluate(y_train_pred, y_train)

    # Print the training loss
    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss}")

    # Make predictions on the test set
    y_test_pred = optimised_ann.forwardPropagation(X_test_scaled)
    test_loss = loss_function.evaluate(y_test_pred, y_test)

    # Print the test loss
    print(f"Test Loss: {test_loss}")

Epoch 1/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 2/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 3/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 4/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 5/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 6/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 7/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 8/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 9/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 10/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 11/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 12/500 - Training Loss: 0.37908212469034924
Test Loss: 0.3434949218047184
Epoch 13/500 - Training Loss: 0.37908212469034924